# Install

In [75]:
# cuda 11.1
#!pip install -r requirements-torch-cu111.txt --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111

In [76]:
import gdown

In [77]:
prefix = 'final'
url = 'https://drive.google.com/uc?id=1akrTOnfP29ycTR7dMZDEPFrprlWZvVAD'

In [78]:
scripts_file = f'data/json.zip'
gdown.download(url, scripts_file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1akrTOnfP29ycTR7dMZDEPFrprlWZvVAD
From (redirected): https://drive.google.com/uc?id=1akrTOnfP29ycTR7dMZDEPFrprlWZvVAD&confirm=t&uuid=5dd12c84-f8ca-4c8b-94cf-4f2777ce4d33
To: c:\Users\sbin0\Desktop\스토리 생성기 mk.2\data\json.zip
100%|██████████| 62.0M/62.0M [00:05<00:00, 10.7MB/s]


'data/json.zip'

In [79]:
import shutil, zipfile

In [80]:
shutil.rmtree('data/json', ignore_errors=True)  # 기존 폴더 삭제
with zipfile.ZipFile('data/json.zip', 'r') as zip_ref:
    zip_ref.extractall('data/json')             # 압축 해제

In [81]:
from glob import glob
import json

In [82]:
json_files = sorted(glob('data/json/*/*.json'))

In [83]:
data_dict = []
for json_file in json_files:
    with open(json_file, encoding='utf-8') as f:
        story_dict = json.load(f)

    story_genre = story_dict.get('genre', ['기타'])  # ⭐ 최상단 genre 가져오기

    for unit in story_dict['units']:
        unit_dict = {}
        unit_dict['uid'] = unit['id']
        unit_dict['storyline'] = unit['storyline']
        unit_dict['genre'] = story_genre  # ⭐ unit에도 장르 복사 (리스트 형태 그대로)
        unit_dict['script'] = [s['content'] for s in unit['story_scripts']]
        data_dict.append(unit_dict)


In [84]:
len(data_dict)

100077

# Data preperation

In [85]:
#prefix = 'final'
#url = 'https://drive.google.com/uc?id=1Bts2h-QPQ5-m7sDIXgVRfumjl-8XHOst'
#url = 'https://drive.google.com/uc?id=1x6HuyJTQcNydJ9P-fJl2LtxnnAu9Vp8N'
#prefix = '1cycle'
#url = 'https://drive.google.com/uc?id=1j46elyFZtkmnmCehlntMi0eX0Tp5nnav'
#prefix = 'helper'
#url = 'https://drive.google.com/uc?id=1iSP_YKFs56d5cRRTEMzfedwRxrx-nXWO'

## 스토리헬퍼 샘플 데이터 다운로드

In [86]:
#scripts_file = f'data/scripts_{prefix}.json'
#zip_file = f'data/scripts_{prefix}.zip'
#gdown.download(url, zip_file, quiet=False)

In [87]:
#!unzip $zip_file 
#!mv -f 'final.json' $scripts_file

In [88]:
#import json

#with open(scripts_file) as f:
#    data_dict = json.load(f)

In [89]:
# 샘플 데이터 출력
data_dict[0]

{'uid': '01_0017_01',
 'storyline': 'C001은 지하철 안에서 졸다가 깨어 내리는데 두 여자아이를 발견하고 섬찟한다.',
 'genre': ['스릴러', '공포(호러)', '드라마'],
 'script': ['C001은 지하철 노약자석 한구석에서 졸고 있다.',
  'C001은 졸다가 눈을 가늘게 떴을 때 어떤 엄마와 아이를 본다.',
  'C001은 열차가 문을 열고 닫는 동안 계속 졸다가 종착역임을 알게 되고 빠져나온다.',
  'C001은 열차 안에 두 여자아이가 잠든 채로 문이 닫히고 출발하는 것을 본다.',
  'C001은 섬찟하지만, 잠에서 막 깨어 피곤해한다.',
  'C001은 집으로 들어가 부엌으로 향한다.',
  '뭐 해?',
  '놀래라. 소리도 없이 들어오네? 귀신인 줄 알았어. 밖에 비 와?',
  'C001은 수건으로 머리를 닦으며 말한다.',
  '제대로 된 우산 하나 있었는데 지난번에 차에 두고 내렸네.',
  '할 수 없네. 갈게. 피곤해 보이니까 잘 쉬고.']}

**후처리**
1. `\n`을 제거한다. "부엌에서 일하게 된 마리오\n인부들 사이에서 인기만점인 베아트리체"  
   ==> 필요없는 것 같음.

In [90]:
# 비정상적 white character가 없는지 확인
for idx, data in enumerate(data_dict):
    #data['storyline'] = data['storyline'].replace('\n', ' ')
    for i, context in enumerate(data['script']):
        #if '\n' in context:
        if '부엌에서 일하게' in context:
            print(idx, i, context)
            print('"%s%s"'%(context[9],context[10]))
            print(context[10] == ' ')
        #data['script'][i] = context.replace('\n', ' ')

## Tokenizer

### kobigbird pretrained model을 이용한 tokenize

In [91]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('monologg/kobigbird-bert-base')


# Word2Vec

In [92]:
import numpy as np

EOS_ID = tokenizer.sep_token_id

positive_sessions = []
positive_str = []
positive_ids = []
for i, unit_data in enumerate(data_dict):
    # 스크립트 토큰화 및 [SEP] 추가
    unit_contexts = [tokenizer.tokenize(text) for text in unit_data['script'] + ['[SEP]']]
    while [] in unit_contexts:
        print('empty string in the script. removing..., id=', i)
        index = unit_contexts.index([])
        del unit_contexts[index]
        del unit_data['script'][index]
    if len(unit_contexts) <= 1:
        print('empty scripts. skipping..., id=', i)
        continue
    # 내러티브 토큰화
    unit_narrative = tokenizer.tokenize(unit_data['storyline'])
    if len(unit_narrative) == 0:
        print('empty narrative. skipping, id=', i)
        continue

    # 장르 정보가 반드시 포함되어야 함 (positive_sessions에만 추가됨)
    if 'genre' not in unit_data or not unit_data['genre']:
        print('no genre info. skipping..., id=', i)
        continue
    # 만약 단일 값이면 리스트로 만들고 여러 값이면 그대로 사용
    genre_info = unit_data['genre'] if isinstance(unit_data['genre'], list) else [unit_data['genre']]

    # positive_sessions 항목에 장르 정보 추가 (항목 개수를 늘리지 않음)
    positive_sessions.append([unit_contexts, unit_narrative, 1, genre_info])
    positive_str.append(unit_data)
    positive_ids.append(unit_data['uid'])

print("all suitable sessions: ", len(positive_sessions))

# reproducibility를 위한 random seed 설정 및 데이터 셔플 (각 리스트 일관되게 셔플)
np.random.seed(42)
np.random.shuffle(positive_sessions)
np.random.seed(42)
np.random.shuffle(positive_str)
np.random.seed(42)
np.random.shuffle(positive_ids)

all suitable sessions:  100077


In [93]:
print(positive_sessions[:5])

[[[['C', '##00', '##2', '##는', '강력', '##팀', '##과', 'C', '##00', '##5', '##에', '##게', '무', '##전', '##한다', '.'], ['CCTV', '상', '나이트', '##클럽', '뒷', '##골목', '##에', '들어갔', '##다가', '차', '##를', '훼손', '##시', '##켜', '##서', '렉', '##카', '##로', '끌', '##고', '나왔', '##습', '##니다', '.'], ['C', '##00', '##8', ',', 'C', '##00', '##9', ',', 'C', '##0', '##16', '##는', '무', '##전', '##을', '듣', '##고', '이동', '##한다', '.'], ['한편', ',', 'C', '##0', '##25', '##와', 'C', '##0', '##12', '부하', '##2', '##는', 'C', '##00', '##1', '##과', 'C', '##0', '##21', '##를', '죽일', '준비', '##를', '한다', '.'], ['형', '##님', ',', '우리', '어떡', '##하', '##죠', '?', '살려', '##주', '##세요', '.', '이러', '##지', '말', '##고', '대화', '##로', '해결', '##해', '##요', '.'], ['C', '##00', '##1', '##은', 'C', '##0', '##12', '부하', '##2', '##를', '부르', '##며', '조롱', '##을', '한다', '.'], ['아까', '네', '##가', '때리', '##는', '게', '너무', '간지', '##러워서', '웃', '##겼', '##어', '.'], ['뭐', '##라고', '?', '이게', '덜', '맞', '##았', '##나', '?'], ['C', '##0', '##25', '##는', 'C', '##0', '##12', '부하'

In [94]:
positive_ids[0:10]

['02_2659_01',
 '03_3732_26',
 '03_1512_14',
 '01_0020_26',
 '01_0338_14',
 '01_2463_17',
 '02_2999_28',
 '01_0068_17',
 '02_0685_04',
 '02_2747_08']

In [95]:
train_num = int(len(positive_sessions) * 0.9)
dev_test_num = int(len(positive_sessions) * 0.05)
train_sessions, dev_sessions, test_sessions = positive_sessions[:train_num], positive_sessions[train_num: train_num + dev_test_num], positive_sessions[train_num + dev_test_num:]
print('number of train =', len(train_sessions), ', val =', len(dev_sessions), ', test =', len(test_sessions))

number of train = 90069 , val = 5003 , test = 5005


In [96]:
train_texts = []
for train_session in train_sessions:
    train_texts += train_session[0]
    train_texts.append(train_session[1])
print('number of word2vec training sentences =', len(train_texts))

number of word2vec training sentences = 1419213


In [97]:
from gensim.models import Word2Vec

# word2vec 학습
model = Word2Vec(sentences = train_texts, vector_size = 200, window = 7, min_count = 5, workers = 4)

In [98]:
print('total num of words =', len(model.wv.key_to_index))
print('first word = "%s"'%model.wv.index_to_key[0])
print('last word = "%s"'%model.wv.index_to_key[-1])

total num of words = 19451
first word = "C"
last word = "재청"


In [99]:
# word2vec이 잘 학습되었는지 여러가지 테스트를 수행하자.
print(model.wv.most_similar("가족"))

[('친구', 0.6844412684440613), ('식구', 0.6206466555595398), ('지인', 0.6158685684204102), ('아이', 0.6108291745185852), ('동생', 0.6014763712882996), ('동료', 0.6002482771873474), ('자녀', 0.6002011299133301), ('백성', 0.594779372215271), ('부대원', 0.5899234414100647), ('사람', 0.5896921753883362)]


## 데이터 저장

`embeddings.pkl`과 `vocab.txt`를 생성한다.

In [100]:
from gensim.models import Word2Vec

# 예시: Word2Vec 모델을 불러왔을 경우
#model = Word2Vec.load(f"data/word2vec_{prefix}.model")

# 이제 .wv 사용 가능
with open(f"data/vocab_{prefix}.txt", "w", encoding="utf-8") as file:
    for i, key in enumerate(model.wv.index_to_key):
        file.write(f"{key}\t{i}\n")


In [101]:
import numpy as np
import pickle

# ✅ 첫 번째 벡터 사이즈를 기반으로 결정
vector_size = model.vector_size
vectors = [np.zeros(vector_size)]

for word in model.wv.index_to_key:
    vec = model.wv.get_vector(word)
    if vec.shape[0] != vector_size:
        print(f"❗ {word} 벡터 차원 불일치: {vec.shape}")
        continue  # 벡터 차원이 안 맞으면 무시
    vectors.append(vec)

# 최종 배열 생성
new_embeddings = np.stack(vectors)

# 저장
with open(f'data/embeddings_{prefix}.pkl', 'wb') as f:
    pickle.dump(new_embeddings, f)


In [102]:
model.save(f"data/word2vec_{prefix}.model")

# 학습 데이터셋 준비

In [103]:
EOS_ID = model.wv.key_to_index['[SEP]'] + 1 if '[SEP]' in model.wv else 1
UNK_ID = model.wv.key_to_index['[UNK]'] + 1 if '[UNK]' in model.wv else 2


In [104]:
import pickle

vocab = {}

with open(f"data/vocab_{prefix}.txt", "r", encoding="utf-8") as fr:
    for idx, line in enumerate(fr):
        line = line.strip().split("\t")
        vocab[line[0]] = idx + 1

In [105]:
# sample id 출력 확인
vocab['가족']

353

**positive data 준비**

In [106]:
# 예시: 장르 사전 만들기
genre_set = set()
for unit in positive_sessions:
    # unit[3]는 기존에 genre_info (리스트 형태)
    for genre in unit[3]:
        genre_set.add(genre)
        
genre_to_id = {genre: idx for idx, genre in enumerate(sorted(genre_set))}
print(genre_to_id)

# positive_data를 만들 때 genre 정보를 숫자 레이블로 변환하여 추가
positive_data = []
positive_str2 = []

for unit, unit_str in zip(positive_sessions, positive_str):
    narrative = unit[1]  # 내러티브 토큰 리스트
    context = unit[0]    # 대화(utterance) 토큰 리스트
    # 문자열로 되어있는 장르 정보를 숫자 리스트로 변환 (여러 개면 리스트 그대로)
    genre_info = [genre_to_id[g] for g in unit[3]]
    
    narrative_id = [vocab.get(word, UNK_ID) for word in narrative]
    context_id = [[vocab.get(word, UNK_ID) for word in sent] for sent in context]

    if len(narrative_id) == 0 or len(context_id) == 0:
        print('empty narrative found. skipping...')
        print(unit_str)
        continue

    # 숫자 레이블로 변환한 genre_info도 데이터에 함께 추가
    data = [context_id, narrative_id, 1, genre_info]
    positive_data.append(data)
    positive_str2.append(unit_str)

print("Total positive_data length:", len(positive_data))


{'SF': 0, '공포(호러)': 1, '드라마': 2, '멜로/로맨스': 3, '미스터리': 4, '스릴러': 5, '액션': 6, '전쟁': 7, '코미디': 8, '판타지': 9}
Total positive_data length: 100077


In [107]:
print(positive_data[:5])

[[[[1, 3, 12, 4, 2692, 1241, 38, 1, 3, 30, 8, 15, 793, 512, 16, 2], [1738, 1053, 5397, 6275, 1508, 8016, 8, 2883, 173, 115, 10, 8094, 150, 1153, 41, 15159, 2219, 37, 797, 14, 1124, 95, 102, 2], [1, 3, 50, 25, 1, 3, 56, 25, 1, 18, 169, 4, 793, 512, 5, 121, 14, 603, 16, 2], [227, 25, 1, 18, 1069, 35, 1, 18, 88, 796, 12, 4, 1, 3, 7, 38, 1, 18, 448, 10, 1708, 299, 10, 46, 2], [386, 127, 25, 128, 925, 22, 135, 21, 1161, 97, 185, 2, 786, 26, 24, 14, 402, 37, 964, 45, 34, 2], [1, 3, 7, 13, 1, 18, 88, 796, 12, 10, 555, 32, 5097, 5, 46, 2], [872, 108, 9, 1249, 4, 139, 229, 7253, 5546, 140, 3385, 28, 2], [122, 69, 21, 599, 2377, 255, 107, 126, 21], [1, 18, 1069, 4, 1, 18, 88, 796, 12, 8, 15, 4980, 8, 2674, 26, 428, 14, 885, 11, 2], [1, 18, 1069, 4, 2485, 7275, 5, 138, 14, 274, 27, 191, 2], [29]], [1, 3, 50, 25, 1, 3, 56, 25, 1, 18, 169, 4, 1, 3, 7, 5, 924, 152, 2609, 63, 1, 3, 7, 13, 1, 18, 88, 796, 12, 10, 4980, 16, 2], 1, [5]], [[[1, 3, 7, 13, 1, 3, 19, 38, 788, 8, 109, 2], [1, 3, 7, 6, 212, 2

In [109]:
len(positive_str), len(positive_str2), len(positive_ids)

(100077, 100077, 100077)

In [110]:
dev_test_num = int(len(positive_data) * 0.05)
train, dev, test = positive_data[:train_num], positive_data[train_num: train_num + dev_test_num], positive_data[train_num + dev_test_num:]
train_ids, dev_ids, test_ids = positive_ids[:train_num], positive_ids[train_num: train_num + dev_test_num], positive_ids[train_num + dev_test_num:]

import random
train_all, dev_all, test_all = [], [], []

# 학습 샘플 생성 (genre_info 추가)
for context_id, narrative_id, _, genre_info in train:
    num_context = len(context_id)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        # 정답(positive) 샘플: 마지막 항목에 genre_info 추가
        train_all.append([context, response, narrative_id, response, 1, genre_info])
        flag = True
        while flag:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if len(response) != len(random_response):
                flag = False
                train_all.append([context, random_response, narrative_id, response, 0, genre_info])
            else:
                for idx, wid in enumerate(response):
                    if wid != random_response[idx]:
                        flag = False
                        train_all.append([context, random_response, narrative_id, response, 0, genre_info])
                        break

print(train_all[0])
print(train_all[1])

# 개발(Validation) 샘플 생성 (genre_info 추가)
dev_all_ids = []
for i_dev, (context_id, narrative_id, _, genre_info) in enumerate(dev):
    num_context = len(context_id)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        dev_all.append([context, response, narrative_id, response, 1, genre_info])
        dev_all_ids.append(dev_ids[i_dev])
        count = 0
        negative_samples = []
        while count < 8:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if random_response not in negative_samples and random_response != [EOS_ID]:
                if len(response) != len(random_response):
                    dev_all.append([context, random_response, narrative_id, response, 0, genre_info])
                    negative_samples.append(random_response)
                    dev_all_ids.append(dev_ids[i_dev])
                    count += 1
                else:
                    for idx, wid in enumerate(response):
                        if wid != random_response[idx]:
                            dev_all.append([context, random_response, narrative_id, response, 0, genre_info])
                            negative_samples.append(random_response)
                            dev_all_ids.append(dev_ids[i_dev])
                            count += 1
                            break
        if response == [EOS_ID]:
            dev_all.append([context, [EOS_ID], narrative_id, response, 1, genre_info])
        else:
            dev_all.append([context, [EOS_ID], narrative_id, response, 0, genre_info])
        dev_all_ids.append(dev_ids[i_dev])

print(dev_all[0], dev_all[1], dev_all[2])

# 테스트 샘플 생성 (genre_info 추가)
test_all = []
test_all_ids = []
test_num_context = []
for i_test, (context_id, narrative_id, _, genre_info) in enumerate(test):
    num_context = len(context_id)
    test_num_context.append(num_context - 1)
    for i in range(1, num_context):
        context = context_id[:i]
        response = context_id[i]
        test_all.append([context, response, narrative_id, response, 1, genre_info])
        test_all_ids.append(test_ids[i_test])
        count = 0
        negative_samples = []
        while count < 8:
            random_idx = random.randint(0, len(positive_data) - 1)
            random_context = positive_data[random_idx][0]
            random_idx_2 = random.randint(0, len(random_context) - 1)
            random_response = random_context[random_idx_2]
            if random_response not in negative_samples and random_response != [EOS_ID]:
                if len(response) != len(random_response):
                    test_all.append([context, random_response, narrative_id, response, 0, genre_info])
                    negative_samples.append(random_response)
                    test_all_ids.append(test_ids[i_test])
                    count += 1
                else:
                    for idx, id in enumerate(response):
                        if id != random_response[idx]:
                            test_all.append([context, random_response, narrative_id, response, 0, genre_info])
                            negative_samples.append(random_response)
                            test_all_ids.append(test_ids[i_test])
                            count += 1
                            break
        if response == [EOS_ID]:
            test_all.append([context, [EOS_ID], narrative_id, response, 1, genre_info])
        else:
            test_all.append([context, [EOS_ID], narrative_id, response, 0, genre_info])
        test_all_ids.append(test_ids[i_test])

print(test_all[0], test_all[1], test_all[2])
print(len(test_all_ids), len(test_all))
print('total train count =', len(train_all))
print('total val count =', len(dev_all))
print('total test count =', len(test_all))
print('Total number of utterances in test:', np.sum(np.array(test_num_context)))

[[[1, 3, 12, 4, 2692, 1241, 38, 1, 3, 30, 8, 15, 793, 512, 16, 2]], [1738, 1053, 5397, 6275, 1508, 8016, 8, 2883, 173, 115, 10, 8094, 150, 1153, 41, 15159, 2219, 37, 797, 14, 1124, 95, 102, 2], [1, 3, 50, 25, 1, 3, 56, 25, 1, 18, 169, 4, 1, 3, 7, 5, 924, 152, 2609, 63, 1, 3, 7, 13, 1, 18, 88, 796, 12, 10, 4980, 16, 2], [1738, 1053, 5397, 6275, 1508, 8016, 8, 2883, 173, 115, 10, 8094, 150, 1153, 41, 15159, 2219, 37, 797, 14, 1124, 95, 102, 2], 1, [5]]
[[[1, 3, 12, 4, 2692, 1241, 38, 1, 3, 30, 8, 15, 793, 512, 16, 2]], [58, 9, 1691, 546, 23, 17, 1278, 90, 155, 39, 58, 9, 1278, 90, 51, 5, 332, 39, 254, 729, 495, 276, 22, 11, 2, 203, 2425, 1118, 9201, 379, 126, 301, 2], [1, 3, 50, 25, 1, 3, 56, 25, 1, 18, 169, 4, 1, 3, 7, 5, 924, 152, 2609, 63, 1, 3, 7, 13, 1, 18, 88, 796, 12, 10, 4980, 16, 2], [1738, 1053, 5397, 6275, 1508, 8016, 8, 2883, 173, 115, 10, 8094, 150, 1153, 41, 15159, 2219, 37, 797, 14, 1124, 95, 102, 2], 0, [5]]
[[[1072, 493, 4, 3049, 3551, 8, 1909, 35, 2023, 33, 3585, 5, 513

In [111]:
def get_numpy_from_nonfixed_2d_array(aa, max_sentence_len=50, max_num_utterance=10, padding_value=0):
    PAD_SEQUENCE = np.array([0] * max_sentence_len)
    rows = np.empty([0, max_sentence_len], dtype='int')
    aa = aa[-max_num_utterance:]
    for a in aa:
        sentence_len = len(a)
        if sentence_len < max_sentence_len:
            rows  = np.append(rows, [np.pad(a, (0, max_sentence_len-sentence_len), 'constant', constant_values=padding_value)[:max_sentence_len]], axis=0)
        else:
            rows = np.append(rows, [a[:max_sentence_len]], axis=0)
    num_utterance = len(aa)
    if num_utterance < max_num_utterance:
        rows = np.append(rows, [PAD_SEQUENCE]*(max_num_utterance-num_utterance), axis=0)
    # add empty +1 sentence
    rows = np.append(rows, [PAD_SEQUENCE], axis=0)
    #return np.concatenate(rows, axis=0).reshape(-1, max_sentence_len)
    return rows

def get_numpy_from_nonfixed_1d_array(a, max_sentence_len=50, padding_value=0):
    sentence_len = len(a)
    if sentence_len < max_sentence_len:
        return np.pad(a, (0, max_sentence_len-sentence_len), 'constant', constant_values=padding_value)
    else:
        return np.array(a[:max_sentence_len])

cc_test_data = [
        [1,2],
        [4,5,6],
        [7]
     ]
#get_numpy_from_nonfixed_2d_array(cc_test_data, max_sentence_len=5, max_num_utterance=4)

In [112]:
from tqdm import tqdm

def pad_process(data, max_sentence_len=50, max_num_utterance=10):
    utterance = []
    response = []
    narrative = []
    gt_response = []
    y_true = []
    genre_info_list = []  # 장르 정보는 그대로 리스트로 보관
    
    for unit in tqdm(data):
        # unit[0]: context (list of utterance token ID lists)
        utterance.append(get_numpy_from_nonfixed_2d_array(unit[0], max_sentence_len=max_sentence_len, max_num_utterance=max_num_utterance))
        # unit[1]: response (list of token IDs)
        response.append(get_numpy_from_nonfixed_1d_array(unit[1], max_sentence_len=max_sentence_len))
        # unit[2]: narrative (list of token IDs)
        narrative.append(get_numpy_from_nonfixed_1d_array(unit[2], max_sentence_len=max_sentence_len))
        # unit[3]: gt_response (list of token IDs)
        gt_response.append(get_numpy_from_nonfixed_1d_array(unit[3], max_sentence_len=max_sentence_len))
        # unit[4]: y_true (정답 여부)
        y_true.append(unit[4])
        # unit[5]: genre_info (숫자로 매핑된 장르 리스트)
        genre_info_list.append(unit[5])
        
    utterance = np.stack(utterance)
    response = np.stack(response)
    narrative = np.stack(narrative)
    gt_response = np.stack(gt_response)
    y_true = np.stack(y_true)
    return (utterance, response, narrative, gt_response, y_true, genre_info_list)
    
# 패딩 적용 (각 데이터셋에 대하여)
train_pad = pad_process(train_all)
dev_pad = pad_process(dev_all)
test_pad = pad_process(test_all)

100%|██████████| 688840/688840 [01:17<00:00, 8869.16it/s]


**학습데이터셋 저장**

In [113]:
with open(f'data/train_{prefix}.pkl', 'wb') as f:
    pickle.dump(train_pad, f)
with open(f'data/dev_{prefix}.pkl', 'wb') as f:
    pickle.dump(dev_pad, f)
with open(f'data/test_{prefix}.pkl', 'wb') as f:
    pickle.dump(test_pad, f)

In [114]:
with open(f'data/positive_{prefix}.pkl', "wb") as f:
    pickle.dump(positive_data, f)
with open(f'data/positive_str_{prefix}.pkl', "wb") as f:
    pickle.dump(positive_str2, f)

In [115]:
with open(f'data/test_all_ids_{prefix}.pkl', "wb") as f:
    pickle.dump(test_all_ids, f)

In [116]:
for unit, unit_str in zip (positive_data, positive_str2):
    len_unit = len(unit[0])
    len_unit_str = len(unit_str['script'])
    if len_unit != len_unit_str+1:
        print(len_unit, len_unit_str)
    #print(unit[0])
    #print(unit_str['script'])
    #break

In [131]:
genre_list = sorted(list(genre_set))
genre_to_id = {genre: i for i, genre in enumerate(genre_list)}
id_to_genre = {v: k for k, v in genre_to_id.items()}

def get_dat(index, data_pad, ids=None):
    # 패딩된 데이터에서 각 항목 추출 (마지막 요소는 장르 정보, 숫자 리스트)
    utterances = data_pad[0][index]
    response    = data_pad[1][index]
    narrative   = data_pad[2][index]
    gt_response = data_pad[3][index]
    y_true      = data_pad[4][index]
    genres      = data_pad[5][index]  # 예: [2, 5]

    # 패딩(0) 제거
    narrative = narrative[narrative != 0]
    response  = response[response != 0]
    gt_response = gt_response[gt_response != 0]

    narrative_str = tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in narrative])
    response_str  = tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in response])
    gt_response_str = tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in gt_response])

    utterance_str = [''] * 10
    for i in range(10):
        utt = utterances[i]
        utt = utt[utt != 0]
        if len(utt) == 0:
            break
        utterance_str[i] = tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in utt])
    
    id_ = ids[index] if ids is not None else None
    
    # 숫자형 장르 정보를 문자열로 변환
    genre_names = [id_to_genre.get(g, "UNK") for g in genres]

    return id_, narrative_str, response_str, gt_response_str, y_true, utterance_str, genre_names

In [134]:
def browse_dat(index, data_pad):
    utterances = data_pad[0][index]
    response    = data_pad[1][index]
    narrative   = data_pad[2][index]
    gt_response = data_pad[3][index]
    y_true      = data_pad[4][index]
    genres      = data_pad[5][index]

    narrative = narrative[narrative != 0]
    response  = response[response != 0]
    gt_response = gt_response[gt_response != 0]

    print('N:', tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in narrative]))
    print('R:', tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in response]))
    print('T:', tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in gt_response]))
    print('Y_true:', y_true)
    
    # 숫자형 장르 정보를 문자열로 변환하여 출력
    genre_names = [id_to_genre.get(g, "UNK") for g in genres]
    print('Genre:', genre_names)
    
    for i in range(10):
        utt = utterances[i]
        utt = utt[utt != 0]
        if len(utt) == 0:
            break
        print('U:', tokenizer.convert_tokens_to_string([model.wv.index_to_key[k - 1] for k in utt]))
    print()


In [135]:
for i in range(110,120): 
    browse_dat(i, test_pad)

N: 고아인 C008은 냄새에 민감한 아이로 자라나. 팔려 간다.
R: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
T: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
Y_true: 1
Genre: ['드라마', '스릴러']
U: 생선을 팔던 한 여인은 아이를 낳았다.
U: 갓난아기는 바닥에 버려진 채 온 힘을 다해서 울음을 터뜨린다.

N: 고아인 C008은 냄새에 민감한 아이로 자라나. 팔려 간다.
R: 마찬가지죠, C001 경사님도 제 뒷조사 열심히 하시던데요?
T: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
Y_true: 0
Genre: ['드라마', '스릴러']
U: 생선을 팔던 한 여인은 아이를 낳았다.
U: 갓난아기는 바닥에 버려진 채 온 힘을 다해서 울음을 터뜨린다.

N: 고아인 C008은 냄새에 민감한 아이로 자라나. 팔려 간다.
R: C001이 호텔 밖으로 이동한다.
T: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
Y_true: 0
Genre: ['드라마', '스릴러']
U: 생선을 팔던 한 여인은 아이를 낳았다.
U: 갓난아기는 바닥에 버려진 채 온 힘을 다해서 울음을 터뜨린다.

N: 고아인 C008은 냄새에 민감한 아이로 자라나. 팔려 간다.
R: 동생은 식당 밖으로 나와 울먹이는 C002를 달래준다.
T: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
Y_true: 0
Genre: ['드라마', '스릴러']
U: 생선을 팔던 한 여인은 아이를 낳았다.
U: 갓난아기는 바닥에 버려진 채 온 힘을 다해서 울음을 터뜨린다.

N: 고아인 C008은 냄새에 민감한 아이로 자라나. 팔려 간다.
R: C002는 먹을 수 있는 것과 그렇지 않은 것을 골라낸다.
T: 신생아를 낳은 여인은 도망치다가 살인자로 몰렸고 사형을 당한다.
Y_true: 0
Genre: ['드라마', '스릴러']
U: 생선을 팔던 한 여인은 아이를 낳았다.
U: 갓난아기는 바닥

In [136]:
import pandas as pd

In [137]:
column_names = ['id', 'Narrative', 'Response', 'GT_Response', 'y_true', 'score', 'R2@1', 'R10@1', 'R10@2', 'R10@5', 'MRR', 'AVG','genre']
for i in range(10):
    column_names.append('U%02d'%(i+1))
print(column_names)
df = pd.DataFrame(columns=column_names)

['id', 'Narrative', 'Response', 'GT_Response', 'y_true', 'score', 'R2@1', 'R10@1', 'R10@2', 'R10@5', 'MRR', 'AVG', 'genre', 'U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08', 'U09', 'U10']


In [138]:
n = len(test_all_ids)
data_dict_all = []
for i in tqdm(range(n)):
    id_, narrative_str, response_str, gt_response_str, y_true, utterance_str, genre = get_dat(i, test_pad, test_all_ids)
    data_dict = { }
    data_dict['id'] = id_
    data_dict['Narrative'] = narrative_str
    data_dict['Response'] = response_str
    data_dict['GT_Response'] = gt_response_str
    data_dict['y_true'] = y_true
    data_dict['genre'] = genre
    for i in range(10):
        data_dict[f'U%02d'%(i+1)] = utterance_str[i]
    #new_row = pd.Series(data_dict)
    #df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
    data_dict_all.append(data_dict)

100%|██████████| 688840/688840 [02:00<00:00, 5703.76it/s]


In [139]:
df = pd.DataFrame.from_dict(data_dict_all)

In [141]:
df.to_excel(f'test_output_{prefix}.xlsx', index=False)

In [142]:
column_names = ['id', 'Narrative', 'Response', 'GT_Response', 'y_true', 'genre']
for i in range(10):
    column_names.append(f'U%02d' % (i + 1))

print(column_names)
df = pd.DataFrame(columns=column_names)

['id', 'Narrative', 'Response', 'GT_Response', 'y_true', 'genre', 'U01', 'U02', 'U03', 'U04', 'U05', 'U06', 'U07', 'U08', 'U09', 'U10']


In [143]:
n = len(dev_all_ids)
data_dict_all = []
for i in tqdm(range(n)):
    id_, narrative_str, response_str, gt_response_str, y_true, utterance_str, genre = get_dat(i, test_pad, test_all_ids)
    
    data_dict = {
        'id': id_,
        'Narrative': narrative_str,
        'Response': response_str,
        'GT_Response': gt_response_str,
        'y_true': y_true,
        'genre': genre  # genre 추가
    }
    for j in range(10):
        data_dict[f'U%02d' % (j + 1)] = utterance_str[j]

    data_dict_all.append(data_dict)

df = pd.DataFrame.from_dict(data_dict_all)
df.to_excel(f'dev_output_{prefix}.xlsx', index=False)


100%|██████████| 686300/686300 [02:02<00:00, 5598.34it/s]
